# Simple evelutionary aglorithm
We took the onemax problem from the DEAP github and adopted it to use neuroGPU for evelotionary algorithm. 


## Imports
We first get the required modules for our evolution.

- `random` gives us a way to generate random bits;
- `base` gives us access to the Toolbox and base Fitness;
- `creator` allows us to create our types;
- `tools` grants us access to the operators bank;
- `algorithms` enables us some ready generic evolutionary loops.

In [12]:
from DEAP_helper import *
from new_score_functions_normalizer_ver import *
import random
import numpy as np
from deap import base, creator, tools, algorithms
# init_nrngpu()
origs = [3.137968,0.983955,0.089259]
mins = [0,0,0]
maxs = [5,5,5]
volts = getVolts(origs)

	Matrix[7] 


## Type Creation
First step with DEAP is to create the required types. Usually the types created are the fitness and the individual. For the OneMax problem, we want to have a solution with as many ones as possible. Thus we need a maximizing fitness and a individual that is a sorted container, we'll choose the standard `list`.

Type creation is done by calling the function `create` in the creator module. This function takes two mandatory arguments and additional optional arguments. The first argument is the actual name of the type that we want to create. The second argument is the base classe that the new type created should inherit from. Finally, the optional arguments are members to add to the new type.

In [13]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

The first line creates a maximizing fitness by replacing, in the base type Fitness, the pure virtual weights attribute by (1.0,) that means to maximize a single objective fitness. The second line creates an Individual class that inherits the properties of list and has a fitness attribute of the type FitnessMax that was just created.

Single objective is considered by DEAP the same way a multi objective function would but with a single value. The weights (as well as the returned value from the evaluation) are **always required** to be iterable.

The created classes are made available in the creator module. We can instantiate directly objects of the created class like follow. This step is not required in an algorithm, the instatiation will be automated later.

In [14]:
ind = creator.Individual([0,0,0])

print(ind)
print(type(ind))
print(type(ind.fitness))

[0, 0, 0]
<class 'deap.creator.Individual'>
<class 'deap.creator.FitnessMax'>


## Toolbox
The toolbox is intended to store functions with their arguments under standard aliases for uses in algorithms. Functions are registered by a function call with two mandatory arguments, the alias to give to the function and the function it will be associate with. Any additional argument will be given as argument when the alias is called.

In [18]:
def initIndv( mins, maxs):
    """
    intialize a random vector in domain defined by mins and maxs
    """
    res = []
    for min1,max1 in zip(mins, maxs):
        res.append(np.random.uniform(min1,max1))
    return res

In [16]:
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", initIndv,mins,maxs)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In the last block of code we created a toolbox object and registered three functions. This first one, `attr_bool`, calls randint from the random module with arguments (0, 1) to create an integer in the interval $[0, 1]$. The second function, `individual`, when called, will use the initRepeat function made available in the tools module to fill an `Individual` class with what is produced by 100 calls to the previously defined `attr_bool` function. The same thing is done for the `population` function.

For example, calling every function individually shows how it proceeds.

In [17]:
bit = toolbox.attr_bool()
ind = toolbox.individual()
pop = toolbox.population(n=1024)

print("bit is of type %s and has value\n%s" % (type(bit), bit))
print("ind is of type %s and contains %d bits\n%s" % (type(ind), len(ind), ind))
print("pop is of type %s and contains %d individuals\n%s" % (type(pop), len(pop), pop))

bit is of type <class 'int'> and has value
0
ind is of type <class 'list'> and contains 3 bits
[4.404455035089229, 2.4621615173900513, 2.4069595265001187]
pop is of type <class 'list'> and contains 1024 individuals
[[3.012544051531484, 2.533719776462662, 4.863180358261494], [2.885818352283734, 4.30734527922243, 3.7820325381235858], [2.6054263441162426, 1.4583533324909648, 4.321697478643775], [4.879538655018372, 4.0669706454473795, 4.209669341128281], [1.134643661792617, 0.6794121594912655, 3.0152268005986453], [0.607229940098577, 0.7779955821824364, 0.19968908596623036], [1.4691934603658696, 2.2744602861189565, 3.2188739115539233], [3.5935886219618873, 0.19023455578877857, 4.66808952780441], [0.05839031484583368, 1.728774510354485, 3.420031074405134], [1.013845376402649, 1.3879862288942524, 3.6024887394110303], [0.7099691241020439, 3.7299246717914674, 2.733735382376401], [1.0920599785670477, 1.139541602767859, 1.2291559101481386], [1.110897151869371, 0.954925505077186, 4.43007041336796

## Evaluation Function
The evaluation function is pretty simple for the OneMax problem, we need to count the number of ones in an individual. We recall here that the returned value must be an iterable of length equal to the number of objectives (weights).

In [36]:
def evalOneMax(individual):
    
    return sum(individual),

## Genetic Operators
Registering the operators and their default arguments in the toolbox is done as follow.

In [37]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.10)
toolbox.register("select", tools.selTournament, tournsize=3)

The evaluation is given the alias `evaluate`. Having a single argument being the individual to evaluate we don’t need to fix any, the individual will be given later in the algorithm. The two points crossover function is registered the same way under the alias `mate`. The mutation, for its part, needs an argument to be fixed (the independent probability of each attribute to be mutated `indpb`). In the algorithms the `mutate()` function is called with the signature `toolbox.mutate(mutant)`. This is the most convenient way because each mutation takes a different number of arguments, having those arguments fixed in the toolbox leave open most of the possibilities to change the mutation (crossover, selection, or evaluation) operator later in your researches. Finally, the selection operator is registered under the name `select` and the size of the tournament set to 3.

We can for example mutate an individual and expect 10% of its attributes to be flipped.

In [38]:
ind = toolbox.individual()
print(ind)
toolbox.mutate(ind)
print(ind)

[0, 0, 0]
[0, 0, 0]


The operators execute their behaviour on the individuals in place, meaning that if the individual is not copied before modified the old individual is lost. Copying an individual is done with the `clone` function available in every toolbox.

In [39]:
mutant = toolbox.clone(ind)
print(mutant is ind)
print(mutant == ind)

False
True


## Evolving the Population
The main program shall be defined in a main function. All previous declarations are put in the global scope of the module to make created objects available for further use in other user experiments.

The main program is very simple. It consists of generating a population, and giving it to the algorithm for it to evolve a solution. Here we will employ some helpful introspection tools such as Statistics and a Hall of Fame. The statistics are computed using `numpy` functions on the population, and the hall of fame keeps track of the best individuals that ever appeared during the evolution. The algorithm take as arguments, among other, the population and the toolbox. Once the evolution is finished the population contains the individuals from the last generation.

In [40]:
def main():
    import numpy
    
    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, stats=stats, halloffame=hof, verbose=True)
    
    return pop, logbook, hof

Next, we protect our call to the main function and launch the evolution, the verbose argument tell to output the stats on every generations. We can print and plot the data returned. 

In [41]:
if __name__ == "__main__":
    pop, log, hof = main()
    print("Best individual is: %s\nwith fitness: %s" % (hof[0], hof[0].fitness))
    
    import matplotlib.pyplot as plt
    gen, avg, min_, max_ = log.select("gen", "avg", "min", "max")
    plt.plot(gen, avg, label="average")
    plt.plot(gen, min_, label="minimum")
    plt.plot(gen, max_, label="maximum")
    plt.xlabel("Generation")
    plt.ylabel("Fitness")
    plt.legend(loc="lower right")
    plt.show()

AttributeError: 'list' object has no attribute 'fitness'